In [1]:
import tensorflow as tf
from numpy.random import multivariate_normal
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

In [2]:
# GENERATE THE DATA

size = 1000
ball_0 = multivariate_normal(mean=[-3,-3], cov=np.eye(2), size=size)
ball_1 = multivariate_normal(mean=[3,3], cov=np.eye(2), size=size)

plt.scatter(ball_0[:,0], ball_0[:,1])
plt.scatter(ball_1[:,0], ball_1[:,1])
plt.show()


# 

In [3]:
def label_to_one_hot(labels):
    enc = OneHotEncoder(categories='auto', sparse=False)
    enc.fit(labels)
    return enc.transform(labels)

In [4]:
y_0 = [0.0] * size 
y_1 = [1.0] * size
y = np.concatenate([y_0, y_1])[:,np.newaxis]
X = np.concatenate([ball_0, ball_1])
y = label_to_one_hot(y)

data = np.hstack([X,y])
print(data.shape)

# permutate random
np.random.shuffle(data)
X = data[:,0:2]
y = data[:,2:]

(2000, 4)


In [5]:
y.shape

(2000, 2)

In [6]:
X.shape

(2000, 2)

In [7]:
NUM_TRAIN = 1500

x_train = X[0:NUM_TRAIN,:]
x_test = X[NUM_TRAIN:,:]

y_train = y[0:NUM_TRAIN]

y_test = y[NUM_TRAIN:]

In [8]:
x_train.shape

(1500, 2)

In [9]:
x_test.shape

(500, 2)

In [10]:
y_test.shape

(500, 2)

In [11]:
NUM_FP = 4
FP_EPS = 0.1
NUM_CLASSES = 2
NUM_FEATURES = 2

# dx is input distortion, number of dxs is equal to number of unique fingerprints
fp_dx = [np.random.rand(1, NUM_FEATURES) * FP_EPS for i in range(NUM_FP)]

# In paper, dy's are the same for each dx, so dy, and they are distortion to logits
# since logits dimension is NUM_CLASSES and we need NUM_CLASSES dy definitions (one
# dy for each class), dy dimension will be NUM_CLASSESS x NUM_CLASSES

# Details in paper, section 2.3 (choosing dys)
fp_dy = -0.2357*np.ones((NUM_CLASSES, NUM_CLASSES))
for cl in range(NUM_CLASSES):
    fp_dy[cl, cl] = 0.7

print('fp_dx')
print(fp_dx)
print('fp_dy')
print(fp_dy)

fp_dx
[array([[0.08884814, 0.03238107]]), array([[0.06215732, 0.04802284]]), array([[0.09207607, 0.0758419 ]]), array([[0.04807036, 0.03846426]])]
fp_dy
[[ 0.7    -0.2357]
 [-0.2357  0.7   ]]


In [12]:
# Fingerprint training example/label pairs

x_train_fp = np.zeros((NUM_FP, NUM_TRAIN, NUM_FEATURES), dtype=np.float32)
for i in range(NUM_FP):
    x_train_fp[i] = x_train + fp_dx[i]

y_train_fp = np.zeros((NUM_FP, NUM_TRAIN, NUM_CLASSES), dtype=np.float32)
for i in range(NUM_FP):
    for j in range(NUM_TRAIN):
        y_train_fp[i][j] = fp_dy[np.argwhere(y_train[j] == float(1))[0][0]]

In [13]:
print(y_train[0:5])
print(y_train_fp[0][0:5])

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[[-0.2357  0.7   ]
 [-0.2357  0.7   ]
 [-0.2357  0.7   ]
 [ 0.7    -0.2357]
 [ 0.7    -0.2357]]


In [27]:
x_tf = tf.placeholder(tf.float32, shape=(None, NUM_FEATURES), name="x")
y_tf = tf.placeholder(tf.float32, shape=(None, NUM_CLASSES), name="y")

x_fp_tf = tf.placeholder(tf.float32, shape=(NUM_FP, None, NUM_FEATURES), name="x_fp")
y_fp_tf = tf.placeholder(tf.float32, shape=(NUM_FP, None, NUM_CLASSES), name="y_fp")

hidden1 = tf.layers.Dense(units=200, activation=tf.nn.relu, name="hidden1")
hidden2 = tf.layers.Dense(units=200, activation=tf.nn.relu, name="hidden2")
out = tf.layers.Dense(units=NUM_CLASSES, activation=tf.nn.relu)

y_logits = hidden1(x_tf)
y_logits = hidden2(y_logits)
y_logits = out(y_logits)
y_logits_normed = y_logits / tf.norm(y_logits, axis=1)
y_preds = tf.nn.softmax(y_logits)

y_fp_preds = []
for i in range(NUM_FP):
    cur_x_fp = x_fp_tf[i]
    cur_y_fp = hidden1(cur_x_fp)
    cur_y_fp = hidden2(cur_y_fp)
    cur_y_fp = out(cur_y_fp)
    cur_y_fp = cur_y_fp / tf.norm(cur_y_fp, axis=1)
    y_fp_preds.append(cur_y_fp)
y_fp_preds = tf.concat(y_fp_preds, axis=0)

loss_vanilla = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_tf, logits=y_logits)
loss_fp = tf.losses.mean_squared_error(y_fp_preds - y_logits_normed, y_fp_tf)
loss = loss_vanilla + 1.0 * loss_fp

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(1000):
  _, loss_value = sess.run((train, loss), feed_dict={x_tf: x_train, y_tf: y_train})
  # print(loss_value)

ValueError: Shapes (4, ?, 2) and (?, 2) are incompatible

In [20]:
final_preds = sess.run(y_pred, feed_dict={x_tf: x_test})
final_preds = final_preds.squeeze(axis=1)
final_preds = final_preds <= 0.5


In [21]:
sns.scatterplot(x=x_test[:,0], y=x_test[:,1], hue=final_preds)
plt.show()

NameError: name 'sns' is not defined

In [3]:
def frange(x, y, jump):
    while x < y:
        yield x
        x += jump

In [4]:
def generate_grid(minX, minY, maxX, maxY, delta):
    pts = []
    for x in frange(minX, maxX, delta):
        for y in frange(minY, maxY, delta):
            pts.append([x, y])
    return np.array(pts)

In [8]:
grid = generate_grid(-5, -5, 5, 5, 0.1)

In [9]:
grid.shape

(10201, 2)